In [1]:
from datetime import date
import os

In [2]:
import sqlite3
import pandas as pd
from contextlib import closing

In [4]:
adresse1 = Address(
    address_id=1,
    street="Mattweg",
    house_number="10",
    city="Arlesheim",
    zip_code="4144",
    country="Schweiz"
)


In [5]:
print(adresse1.get_address())
print(adresse1.street)

In [6]:
buchung = Bookings(
    booking_id=1,
    check_in_date="2025-06-01",
    check_out_date="2025-06-07",
    is_cancelled=False,
    booking_date="2025-04-21",
    total_amount=399.00,
    room_id=305,
    guest_id=1234,
    guest_name="Max Muster")

In [49]:
print(buchung.get_booking_summary())

In [52]:
buchung.cancel_booking()
print(buchung.get_booking_summary())

In [317]:
class Bookings:
    __next_id = 1
    def __init__(self, check_in_date, check_out_date, is_cancelled, booking_date, total_amount, room_id, guest, payment_id=None, currency_id=None):
        self.__booking_id = Bookings.__next_id
        Bookings.__next_id += 1
        self.__check_in_date = check_in_date
        self.__check_out_date = check_out_date
        self.__is_cancelled = is_cancelled
        self.__booking_date = booking_date
        self.__total_amount = total_amount
        self.__original_total_amount = total_amount
        self.__room_id = room_id
        self.__guest = guest
        self.__payment_id = payment_id
        self.__currency_id = currency_id

    @property
    def booking_id(self):
        return self.__booking_id

    @property
    def check_in_date(self):
        return self.__check_in_date

    @property
    def check_out_date(self):
        return self.__check_out_date

    @property
    def is_cancelled(self):
        return self.__is_cancelled

    @property
    def booking_date(self):
        return self.__booking_date

    @property
    def total_amount(self):
        return self.__total_amount
    
    @property
    def original_total_amount(self):
        return self.__original_total_amount

    @property
    def available_points(self):
        return self.__guest.loyalty_points

    @property
    def applied_points(self):
        return min(self.__total_amount, self.available_points)

    @property
    def room_id(self):
        return self.__room_id

    @property
    def guest(self):
        return self.__guest
    
    @property
    def payment_id(self):
        return self.__payment_id

    @property
    def currency_id(self):
        return self.__currency_id

    def cancel_booking(self):
        self.__is_cancelled = True
    
    def get_booking_summary(self):
        storniert = "Ja" if self.__is_cancelled else "Nein"
        return f"Buchung {self.__booking_id}: {self.__guest.get_full_name()} - {self.__check_in_date} bis {self.__check_out_date} - Zimmer {self.__room_id} - Betrag: CHF {self.__total_amount} - Storniert: {storniert}"
    
    def apply_loyalty_points_from_guest(self):
        if self.applied_points <= 0:
            print("Keine Treuepunkte verfügbar.")
            return

        used_points = self.applied_points
        self.__guest.deduct_points(used_points)
        self.__total_amount -= used_points
        print(f"{used_points} Punkte verwendet. Neuer Betrag: CHF {self.__total_amount:.2f}")



    def add_loyalty_points_to_guest(self):
        if self.original_total_amount < 100:
            print("Keine Treuepunkte hinzugefügt, da der Mindestbetrag 100 CHF beträgt.")
            return
        points = int(self.original_total_amount // 10)  
        self.__guest.add_points(points)
        print(f"{points} Punkte hinzugefügt. Neuer Punktestand: {self.__guest.loyalty_points}")


In [269]:
print(Booking8.applied_points)
print(Booking8.total_amount)
print(Booking8.total_amount_discounted)
print(Booking8.guest.deduct_points(Booking8.applied_points))
print(Booking8.guest.loyalty_points)

In [320]:
Booking10 = Bookings(
    check_in_date="2025-06-02",
    check_out_date="2025-06-08",
    is_cancelled=False,
    booking_date="2025-04-21",
    total_amount=19.00,
    room_id=305,
    guest=guest4
)
Booking10.apply_loyalty_points_from_guest()
Booking10.add_loyalty_points_to_guest()
print(Booking10.get_booking_summary())
print(f"Treuepunkte des Gastes: {Booking10.guest.loyalty_points}")

In [308]:
print(guest4.loyalty_points)

In [245]:
class Guests:
    __next_id = 1  

    def __init__(self,first_name, last_name, email, phone_number, address, loyalty_points=0, booking_history=None):
        self.__guest_id = Guests.__next_id
        Guests.__next_id += 1  
        self.__first_name = first_name
        self.__last_name = last_name
        self.__email = email
        self.__phone_number = phone_number
        self.__address = address
        self.__loyalty_points = loyalty_points
        self.__booking_history = booking_history if booking_history is not None else []
        
    @property
    def guest_id(self):
        return self.__guest_id
        
    @property
    def first_name(self):
        return self.__first_name

    @property
    def last_name(self):
        return self.__last_name

    @property
    def email(self):
        return self.__email

    @property
    def phone_number(self):
        return self.__phone_number

    @property
    def address(self):
        return self.__address

    @property
    def loyalty_points(self):
        return self.__loyalty_points

    def get_full_name(self):
        return f"{self.__first_name} {self.__last_name}"

    def deduct_points(self, points):
        if points > self.__loyalty_points:
            print("Nicht genügend Treuepunkte vorhanden.")
            return
        self.__loyalty_points -= points

    def add_points(self, points):
        self.__loyalty_points += points

    def get_guest_summary(self):
        return f"{self.__guest_id} - Name: {self.__first_name} {self.__last_name} - Email: {self.__email} - Telefonnummer: {self.__phone_number} - Adresse: {self.__address} - Treuepunkte: {self.__loyalty_points}"

In [19]:
class Address:
    __next_id = 1 

    def __init__(self, street, house_number, city, zip_code, country):
        self.__address_id = Address.__next_id
        Address.__next_id += 1 
        self.__street = street
        self.__house_number = house_number
        self.__city = city
        self.__zip_code = zip_code
        self.__country = country

    @property
    def address_id(self):
        return self.__address_id

    @property
    def street(self):
        return self.__street

    @property
    def house_number(self):
        return self.__house_number

    @property
    def city(self):
        return self.__city

    @property
    def zip_code(self):
        return self.__zip_code

    @property
    def country(self):
        return self.__country

    def get_address_summary(self):
        return f"{self.__address_id} - Strasse: {self.__street} {self.__house_number}, {self.__zip_code} {self.__city}, {self.__country}"


In [22]:
address3 = Address(
    street="Mustxerstraße",
    house_number="1",
    city="Musterstadt",
    zip_code="12345",
    country="Musterland"
)

print(address3.get_address_summary())

In [290]:
guest4 = Guests(
    first_name="ax",
    last_name="Mustermann",
    email="max.mustermann@me.com",
    phone_number="0123456789",
    address=address3,
    loyalty_points=200,
    booking_history=[]
)

print(guest4.get_guest_summary())

In [30]:
print(guest2.loyalty_points)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=429e0553-6499-4e09-987f-1a124f4e7b05' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>